In [ ]:
from pykitPIV.ml import PIVEnv
import matplotlib.pyplot as plt
import cmcrameri.cm as cmc

Prepare specs for pykitPIV parameters:

In [ ]:
particle_spec = {'diameters': (1, 1),
                 'distances': (2, 2),
                 'densities': (0.2, 0.2),
                 'diameter_std': 1,
                 'seeding_mode': 'random'},

flowfield_spec = {'gaussian_filters': (30,30),
                  'n_gaussian_filter_iter': 5,
                  'displacement': (2,2)},

motion_spec = {'n_steps': 10,
               'time_separation': 5,
               'particle_loss': (0, 2),
               'particle_gain': (0, 2)},

image_spec = {'exposures': (0.5, 0.9),
              'maximum_intensity': 2**16-1,
              'laser_beam_thickness': 1,
              'laser_over_exposure': 1,
              'laser_beam_shape': 0.95,
              'alpha': 1/8,
              'clip_intensities': True,
              'normalize_intensities': False},

Initialize the `Gymnasium` environment:

In [ ]:
env = PIVEnv(interrogation_window_size=(100,200),
             interrogation_window_size_buffer=10,
             particle_spec=particle_spec,
             flowfield_size=(500,1000),
             flowfield_spec=flowfield_spec,
             motion_spec=motion_spec,
             image_spec=image_spec,
             user_flowfield=None,
             random_seed=100)

In [ ]:
env.observation_space.sample()

In [ ]:
camera_position = env.observation_space.sample()

In [ ]:
camera_position

In [ ]:
env.render(camera_position,
           c='white',
           s=20,
           lw=1,
           figsize=(12,6))

In [ ]:
image_obj = env.record_particles(camera_position)

In [ ]:
plt.imshow(image_obj.images_I1[0,0,:,:], cmap='Greys_r', origin='lower')

In [ ]:
plt.imshow(image_obj.images_I2[0,0,:,:], cmap='Greys_r', origin='lower')

In [ ]:
image_obj.plot_field_magnitude(0, 
                               add_streamplot=True,
                               streamplot_density=1,
                               streamplot_color='k',)

In [ ]:
image_obj.animate_image_pair(0, filename='image.gif')

***